In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import time
import logging

logging.basicConfig(level=logging.INFO)

nse_tickers = [
    "DMART.NS","TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS", "HDFCLIFE.NS",
    "MAXHEALTH.NS", "SBILIFE.NS", "TRENT.NS", "VBL.NS", "KPIL.NS", "GMRPUI.NS","SWIGGY.NS","ETERNAL.NS","PAYTM.NS","ADANIPOWER.NS","HEG.NS","PAYTM.NS"
]

def safe_scalar(val):
    try:
        if isinstance(val, pd.Series):
            if not val.empty:
                return float(val.iloc[0])
            else:
                return np.nan
        else:
            return float(val)
    except:
        return np.nan

def fetch_yfinance_data(ticker):
    return yf.download(ticker, period="90d", interval="1d", auto_adjust=True)

def calculate_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -1 * delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_ma(df, windows=[10, 20, 50]):
    for w in windows:
        df[f'MA_{w}'] = df['Close'].rolling(window=w).mean()
    return df

def find_swing_zones(df, order=5):
    local_min_idx = argrelextrema(df['Close'].values, np.less_equal, order=order)[0]
    demand_zones = df.iloc[local_min_idx]

    local_max_idx = argrelextrema(df['Close'].values, np.greater_equal, order=order)[0]
    supply_zones = df.iloc[local_max_idx]

    return demand_zones, supply_zones

def calc_support_resistance_zone(df, window=10, pct_range=0.02):
    recent_data = df.tail(window)
    low = safe_scalar(recent_data['Low'].min())
    high = safe_scalar(recent_data['High'].max())
    support_zone_low = low
    support_zone_high = low * (1 + pct_range) if pd.notna(low) else np.nan
    resistance_zone_low = high * (1 - pct_range) if pd.notna(high) else np.nan
    resistance_zone_high = high
    return support_zone_low, support_zone_high, resistance_zone_low, resistance_zone_high

def generate_score(rsi, ma_10, ma_20, ma_50, volume, volavg20):
    score = 0
    reasons = []
    if float(rsi) < 30:
        score += 2
        reasons.append('RSI oversold')
    elif float(rsi) > 70:
        score -= 2
        reasons.append('RSI overbought')
    elif 45 <= float(rsi) <= 55:
        score += 1
        reasons.append('RSI neutral strong')

    if float(ma_10) > float(ma_20) and float(ma_20) > float(ma_50):
        score += 2
        reasons.append('MA 10 > MA 20 > MA 50 strong uptrend')
    elif float(ma_10) < float(ma_20) and float(ma_20) < float(ma_50):
        score -= 2
        reasons.append('MA 10 < MA 20 < MA 50 strong downtrend')

    if float(volume) > 1.5 * float(volavg20):
        score += 1
        reasons.append('High volume')

    if score >= 3:
        signal = 'BUY'
    elif score <= -2:
        signal = 'SELL'
    else:
        signal = 'HOLD'

    if not reasons:
        reasons.append('No strong signals')

    return score, signal, '; '.join(reasons)

def calc_trade_levels(price, demand_zone, supply_zone, support_low, support_high, resistance_low, resistance_high, signal):
    if signal != 'BUY':
        # No trade recommended for SELL or HOLD signals
        return None, None, None, None, None

    # For BUY signals:
    entry = price if price > demand_zone else demand_zone if demand_zone else price
    stop_loss = demand_zone * 0.98 if demand_zone else price * 0.98

    # Targets above entry price
    targets = []
    if resistance_low and resistance_low > entry:
        targets.append(resistance_low)
    if resistance_high and resistance_high > entry:
        targets.append(resistance_high)
    if supply_zone and supply_zone > entry:
        targets.append(supply_zone)

    while len(targets) < 3:
        targets.append(None)

    return round(entry, 2), round(stop_loss, 2), round(targets[0], 2) if targets[0] else None, round(targets[1], 2) if targets[1] else None, round(targets[2], 2) if targets[2] else None


results = []

for ticker in nse_tickers:
    logging.info(f'Processing {ticker}...')
    df = fetch_yfinance_data(ticker)
    if df.empty:
        logging.warning(f'No data for {ticker}, skipping.')
        continue

    df = calculate_ma(df)
    df['RSI'] = calculate_rsi(df)
    df['VolAvg20'] = df['Volume'].rolling(20).mean()

    demand_zones, supply_zones = find_swing_zones(df, order=5)

    latest = df.iloc[-1]
    last_date = latest.name

    demand_recent = demand_zones[demand_zones.index < last_date]
    demand_zone_price = safe_scalar(demand_recent['Close'].iloc[-1]) if not demand_recent.empty else np.nan
    demand_zone_val = round(float(demand_zone_price), 2) if pd.notna(demand_zone_price) else None

    supply_recent = supply_zones[supply_zones.index < last_date]
    supply_zone_price = safe_scalar(supply_recent['Close'].iloc[-1]) if not supply_recent.empty else np.nan
    supply_zone_val = round(float(supply_zone_price), 2) if pd.notna(supply_zone_price) else None

    support_zone_low, support_zone_high, resistance_zone_low, resistance_zone_high = calc_support_resistance_zone(df)

    price_val = safe_scalar(latest['Close'])
    rsi_val = safe_scalar(latest['RSI'])
    ma_10_val = safe_scalar(latest['MA_10'])
    ma_20_val = safe_scalar(latest['MA_20'])
    ma_50_val = safe_scalar(latest['MA_50'])
    volume_val = safe_scalar(latest['Volume'])
    volavg20_val = safe_scalar(latest['VolAvg20'])

    score, signal, reasons = generate_score(rsi_val, ma_10_val, ma_20_val, ma_50_val, volume_val, volavg20_val)

    entry, stop_loss, target1, target2, target3 = calc_trade_levels(
        price_val, demand_zone_val, supply_zone_val,
        support_zone_low, support_zone_high,
        resistance_zone_low, resistance_zone_high,
        signal
    )

    result = {
        'Ticker': ticker.replace('.NS',''),
        'Price': round(price_val, 2) if pd.notna(price_val) else None,
        'RSI': round(rsi_val, 2) if pd.notna(rsi_val) else None,
        'MA_10': round(ma_10_val, 2) if pd.notna(ma_10_val) else None,
        'MA_20': round(ma_20_val, 2) if pd.notna(ma_20_val) else None,
        'MA_50': round(ma_50_val, 2) if pd.notna(ma_50_val) else None,
        'Volume': int(volume_val) if pd.notna(volume_val) else None,
        'VolAvg20': int(volavg20_val) if pd.notna(volavg20_val) else None,
        'SupportZoneLow': round(support_zone_low, 2) if pd.notna(support_zone_low) else None,
        'SupportZoneHigh': round(support_zone_high, 2) if pd.notna(support_zone_high) else None,
        'ResistanceZoneLow': round(resistance_zone_low, 2) if pd.notna(resistance_zone_low) else None,
        'ResistanceZoneHigh': round(resistance_zone_high, 2) if pd.notna(resistance_zone_high) else None,
        'DemandZone': demand_zone_val,
        'SupplyZone': supply_zone_val,
        'Entry': entry,
        'StopLoss': stop_loss,
        'Target1': target1,
        'Target2': target2,
        'Target3': target3,
        'Score': score,
        'Signal': signal,
        'Reasons': reasons
    }

    results.append(result)
    time.sleep(1)  # Respect API limits

df_out = pd.DataFrame(results)
print(df_out)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: GMRPUI.NS"}}}
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed 

        Ticker    Price    RSI    MA_10    MA_20    MA_50     Volume  \
0        DMART  4576.20  34.57  4674.38  4699.52  4467.31     454331   
1          TCS  2957.40  39.88  3103.87  3094.58  3090.40    4971744   
2          BEL   403.15  72.05   404.11   390.00   386.58   23800929   
3          DLF   721.05  34.28   765.60   759.77   774.74    4484651   
4     GODREJCP  1189.20  37.31  1225.70  1239.50  1232.31    1908827   
5     HDFCLIFE   765.05  52.83   776.42   772.89   768.89    2080889   
6    MAXHEALTH  1137.70  40.84  1160.47  1162.83  1209.25    5021970   
7      SBILIFE  1809.80  50.94  1824.27  1815.70  1826.60    1003675   
8        TRENT  4742.50   7.88  5037.30  5198.38  5259.96    1517967   
9          VBL   451.85  34.36   465.03   474.77   491.72    8205942   
10        KPIL  1255.10  46.90  1269.56  1264.65  1225.97     110720   
11      SWIGGY   426.10  43.32   438.98   433.71   417.62    8852879   
12     ETERNAL   332.25  53.18   332.29   328.48   313.75   1665

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import mplfinance as mpf
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display, clear_output

def safe_scalar(val):
    try:
        if isinstance(val, pd.Series):
            if not val.empty:
                return float(val.iloc[0])
            else:
                return np.nan
        else:
            return float(val)
    except:
        return np.nan

def calculate_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = (-delta).clip(lower=0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_ma(df, windows=[10, 20, 50]):
    for w in windows:
        df[f'MA_{w}'] = df['Close'].rolling(window=w).mean()
    return df

def find_swing_zones(df, order=5):
    local_min_idx = argrelextrema(df['Close'].values, np.less_equal, order=order)[0]
    demand_zones = df.iloc[local_min_idx]
    local_max_idx = argrelextrema(df['Close'].values, np.greater_equal, order=order)[0]
    supply_zones = df.iloc[local_max_idx]
    return demand_zones, supply_zones

def calc_support_resistance_zone(df, window=10, pct_range=0.02):
    recent_data = df.tail(window)
    low = safe_scalar(recent_data['Low'].min())
    high = safe_scalar(recent_data['High'].max())
    support_zone_low = low
    support_zone_high = low * (1 + pct_range) if pd.notna(low) else np.nan
    resistance_zone_low = high * (1 - pct_range) if pd.notna(high) else np.nan
    resistance_zone_high = high
    return support_zone_low, support_zone_high, resistance_zone_low, resistance_zone_high

def generate_score(rsi, ma_10, ma_20, ma_50, volume, volavg20):
    score = 0
    reasons = []
    if float(rsi) < 30:
        score += 2
        reasons.append('RSI oversold')
    elif float(rsi) > 70:
        score -= 2
        reasons.append('RSI overbought')
    elif 45 <= float(rsi) <= 55:
        score += 1
        reasons.append('RSI neutral strong')

    if float(ma_10) > float(ma_20) and float(ma_20) > float(ma_50):
        score += 2
        reasons.append('MA 10 > MA 20 > MA 50 strong uptrend')
    elif float(ma_10) < float(ma_20) and float(ma_20) < float(ma_50):
        score -= 2
        reasons.append('MA 10 < MA 20 < MA 50 strong downtrend')

    if float(volume) > 1.5 * float(volavg20):
        score += 1
        reasons.append('High volume')

    if score >= 3:
        signal = 'BUY'
    elif score <= -2:
        signal = 'SELL'
    else:
        signal = 'HOLD'

    if not reasons:
        reasons.append('No strong signals')

    return score, signal, '; '.join(reasons)

def plot_stock_analysis_with_rsi_colab(
        ticker, period="6mo", interval="1d",
        demand_zone_val=None, supply_zone_val=None,
        support_zone_low=None, support_zone_high=None,
        resistance_zone_low=None, resistance_zone_high=None,
        signal=""):

    df = yf.download(ticker, period=period, interval=interval, group_by="column", auto_adjust=True)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0] for col in df.columns]
    df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)

    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    addplots = [
        mpf.make_addplot(df['RSI'], panel=2, color='purple', ylabel='RSI'),
        mpf.make_addplot([70] * len(df), panel=2, color='red', linestyle='--'),
        mpf.make_addplot([30] * len(df), panel=2, color='green', linestyle='--')
    ]

    mavcolors = ['blue', 'orange', 'green']
    fig, axes = mpf.plot(
        df, type='candle', mav=(10, 20, 50), volume=True,
        addplot=addplots, panel_ratios=(6, 2, 3), style='yahoo',
        title=f"{ticker} | Signal: {signal}",
        ylabel='Price', ylabel_lower='Volume',
        figratio=(30, 10),  # Wider figure width here
        figscale=1.35,          # Larger scale for size
        mavcolors=mavcolors,
        returnfig=True
    )

    x = np.arange(len(df))

    if demand_zone_val is not None and pd.notna(demand_zone_val):
        axes[0].fill_between(x, demand_zone_val * 0.98, demand_zone_val * 1.02,
                             color='green', alpha=0.2, label="Demand Zone")
    if supply_zone_val is not None and pd.notna(supply_zone_val):
        axes[0].fill_between(x, supply_zone_val * 0.98, supply_zone_val * 1.02,
                             color='red', alpha=0.2, label="Supply Zone")

    if support_zone_low is not None and support_zone_high is not None \
            and pd.notna(support_zone_low) and pd.notna(support_zone_high):
        axes[0].fill_between(x, support_zone_low, support_zone_high,
                             color='green', alpha=0.18, label="Support Zone Band")
        axes[0].hlines([support_zone_low, support_zone_high], xmin=0, xmax=len(df),
                       colors='green', linestyles='--', linewidth=1.5, label="Support Zone")

    if resistance_zone_low is not None and resistance_zone_high is not None \
            and pd.notna(resistance_zone_low) and pd.notna(resistance_zone_high):
        axes[0].fill_between(x, resistance_zone_low, resistance_zone_high,
                             color='red', alpha=0.18, label="Resistance Zone Band")
        axes[0].hlines([resistance_zone_low, resistance_zone_high], xmin=0, xmax=len(df),
                       colors='red', linestyles='--', linewidth=1.5, label="Resistance Zone")

    handles = [
        mpatches.Patch(color='blue', label='MA 10'),
        mpatches.Patch(color='orange', label='MA 20'),
        mpatches.Patch(color='green', label='MA 50')
    ]
    axes[0].legend(handles=handles, loc='upper left')

    last_close = round(df['Close'].iloc[-1], 2)
    last_rsi = round(df['RSI'].iloc[-1], 1)
    info_text = (f"Price: {last_close}   |   RSI: {last_rsi}   |   "
                 f"Support Zone: [{support_zone_low}, {support_zone_high}]   |   "
                 f"Resistance Zone: [{resistance_zone_low}, {resistance_zone_high}]"
                 f" Demand Zone: [{demand_zone_val}]   |   "
                 f"Supply Zone: [{supply_zone_val}]")
    axes[0].text(0.01, 1.02, info_text, transform=axes[0].transAxes,
                 fontsize=10, va='bottom', ha='left',
                 bbox=dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.8))

    import matplotlib.pyplot as plt
    plt.show()

def run_analysis(tickers):
    ticker_data = {}
    for ticker in tickers:
        print(f"Processing {ticker} ...")
        df = yf.download(ticker, period="90d", interval="1d", auto_adjust=True)
        if df.empty:
            print(f"No data for {ticker}, skipping.")
            continue

        df = calculate_ma(df)
        df['RSI'] = calculate_rsi(df)
        demand_zones, supply_zones = find_swing_zones(df)
        last_date = df.index[-1]

        demand_recent = demand_zones[demand_zones.index < last_date]
        demand_zone_price = safe_scalar(demand_recent['Close'].iloc[-1]) if not demand_recent.empty else None
        demand_zone_val = round(float(demand_zone_price), 2) if pd.notna(demand_zone_price) else None

        supply_recent = supply_zones[supply_zones.index < last_date]
        supply_zone_price = safe_scalar(supply_recent['Close'].iloc[-1]) if not supply_recent.empty else None
        supply_zone_val = round(float(supply_zone_price), 2) if pd.notna(supply_zone_price) else None

        support_zone_low, support_zone_high, resistance_zone_low, resistance_zone_high = calc_support_resistance_zone(df)

        price_val = safe_scalar(df['Close'].iloc[-1])
        rsi_val = safe_scalar(df['RSI'].iloc[-1])
        ma_10_val = safe_scalar(df['MA_10'].iloc[-1])
        ma_20_val = safe_scalar(df['MA_20'].iloc[-1])
        ma_50_val = safe_scalar(df['MA_50'].iloc[-1])
        volume_val = safe_scalar(df['Volume'].iloc[-1])
        volavg20_val = safe_scalar(df['Volume'].rolling(20).mean().iloc[-1])
        score, detected_signal, reasons = generate_score(
            rsi_val, ma_10_val, ma_20_val, ma_50_val, volume_val, volavg20_val)

        ticker_data[ticker] = {
            'df': df,
            'demand_zone_val': demand_zone_val,
            'supply_zone_val': supply_zone_val,
            'support_zone_low': support_zone_low,
            'support_zone_high': support_zone_high,
            'resistance_zone_low': resistance_zone_low,
            'resistance_zone_high': resistance_zone_high,
            'signal': detected_signal
        }
    print("Analysis complete.")
    return ticker_data

def interactive_plot(ticker_data):
    tickers_list = list(ticker_data.keys())
    dropdown = widgets.Combobox(
        placeholder='Type or select ticker',
        options=tickers_list,
        description='Ticker:',
        ensure_option=True,
        continuous_update=False,
        layout=widgets.Layout(width='300px')
    )
    output = widgets.Output()

    def on_ticker_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_ticker = change['new']
            if selected_ticker in ticker_data:
                clear_output(wait=True)
                with output:
                    data = ticker_data[selected_ticker]
                    plot_stock_analysis_with_rsi_colab(
                        ticker=selected_ticker,
                        demand_zone_val=data['demand_zone_val'],
                        supply_zone_val=data['supply_zone_val'],
                        support_zone_low=data['support_zone_low'],
                        support_zone_high=data['support_zone_high'],
                        resistance_zone_low=data['resistance_zone_low'],
                        resistance_zone_high=data['resistance_zone_high'],
                        signal=data['signal']
                    )

    dropdown.observe(on_ticker_change)
    display(dropdown, output)

# List your tickers below
nse_tickers = [
    "DMART.NS","TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS", "HDFCLIFE.NS",
    "MAXHEALTH.NS", "SBILIFE.NS", "TRENT.NS", "VBL.NS", "KPIL.NS",
    "GMRPUI.NS", "SWIGGY.NS", "ETERNAL.NS", "PAYTM.NS",
    "ADANIPOWER.NS", "HEG.NS"
]

# Run and cache results for all tickers
ticker_results = run_analysis(nse_tickers)

# Launch the interactive dropdown selector and plot viewer
interactive_plot(ticker_results)


[*********************100%***********************]  1 of 1 completed

Processing DMART.NS ...
Processing TCS.NS ...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Processing BEL.NS ...
Processing DLF.NS ...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

Processing HEG.NS ...


[*********************100%***********************]  1 of 1 completed

Analysis complete.


Combobox(value='', continuous_update=False, description='Ticker:', ensure_option=True, layout=Layout(width='30…

Output()

In [3]:
pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.3 MB/s eta 0:00:00


**Main Summary of Backtester 3**

Here some small cahnges i have made that is I have triggereed SELL signal as HOLD or SELL

In [35]:
import backtrader as bt
import yfinance as yf
import pandas as pd


class CustomStrategy(bt.Strategy):
    params = dict(rsi_period=14, ma_periods=(10, 20, 50), vol_period=20)

    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)
        self.ma10 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[0])
        self.ma20 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[1])
        self.ma50 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[2])
        self.volma = bt.indicators.SMA(self.data.volume, period=self.p.vol_period)

    def next(self):
        score = 0
        if self.rsi[0] < 30:
            score += 2
        elif self.rsi[0] > 70:
            score -= 2
        elif 45 <= self.rsi[0] <= 55:
            score += 1

        if self.ma10[0] > self.ma20[0] > self.ma50[0]:
            score += 2
        elif self.ma10[0] < self.ma20[0] < self.ma50[0]:
            score -= 2

        if self.data.volume[0] > 1.5 * self.volma[0]:
            score += 1

        if score >= 3:
            signal = "BUY"
        elif score <=  -2:
            signal = "SELL"
        else:
            signal = "HOLD"

        if not self.position:
            if signal == "BUY":
                self.buy()
        else:
            if signal in ("SELL", "HOLD"):
                self.sell()


tickers = [
    "DMART.NS", "TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS",
    "HDFCLIFE.NS", "MAXHEALTH", "SBIL", "TRENT.NS", "VBL.NS",
    "KPIL", "GMRTHU", "SWIGGY", "ETERNAL", "PAYTM", "ADANPOWER", "HEG"
]

results = []

for ticker in tickers:
    print(f"Running backtest on {ticker}")
    data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
    if data.empty:
        print(f"No data for {ticker}, skipping.")
        continue

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(1)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(CustomStrategy)
    data_feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_feed)
    cerebro.broker.setcash(10000)

    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade")

    strat = cerebro.run()[0]

    trade_analysis = strat.analyzers.trade.get_analysis()

    total_trades = trade_analysis.get("total", {}).get("closed", 0)
    won_trades = trade_analysis.get("won", {}).get("total", 0)
    lost_trades = trade_analysis.get("lost", {}).get("total", 0)
    win_rate = won_trades / total_trades if total_trades != 0 else 0
    gross_profit = trade_analysis.get("won", {}).get("pnl", {}).get("total", 0)
    gross_loss = trade_analysis.get("lost", {}).get("pnl", {}).get("total", 0)
    net_profit = trade_analysis.get("pnl", {}).get("net", {}).get("total", 0)
    avg_win = trade_analysis.get("won", {}).get("pnl", {}).get("average", 0)
    avg_loss = trade_analysis.get("lost", {}).get("pnl", {}).get("average", 0)
    avg_trade_pnl = net_profit / total_trades if total_trades != 0 else 0
    max_win = trade_analysis.get("won", {}).get("pnl", {}).get("max", 0)
    max_loss = trade_analysis.get("lost", {}).get("pnl", {}).get("min", 0)
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float("inf")

    max_drawdown = strat.analyzers.drawdown.get_analysis().max.drawdown
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get("sharperatio", None)
    sqn = strat.analyzers.sqn.get_analysis().get("sqn", None)

    results.append(
        {
            "ticker": ticker,
            "final_value": cerebro.broker.getvalue(),
            "sharpe": sharpe_ratio,
            "sqn": sqn,
            "max_drawdown": max_drawdown,
            "total_trades": total_trades,
            "winning_trades": won_trades,
            "losing_trades": lost_trades,
            "win_rate": win_rate,
            "gross_profit": gross_profit,
            "gross_loss": gross_loss,
            "net_profit": net_profit,
            "average_win": avg_win,
            "average_loss": avg_loss,
            "average_trade_pnl": avg_trade_pnl,
            "max_win": max_win,
            "max_loss": max_loss,
            "profit_factor": profit_factor,
        }
    )

df = pd.DataFrame(results)

# Calculate aggregates
mean_cols = [
    "final_value",
    "sqn",
    "max_drawdown",
    "win_rate",
    "gross_profit",
    "gross_loss",
    "average_win",
    "average_loss",
    "average_trade_pnl",
]
sum_cols = ["total_trades", "winning_trades", "losing_trades", "net_profit"]
max_cols = ["max_win", "max_loss"]

aggregate_summary = pd.DataFrame(
    {
        "Mean": df[mean_cols].mean(),
        "Sum": df[sum_cols].sum(),
        "Max": df[max_cols].max(),
    }
)

print("Aggregate Summary:")
print(aggregate_summary.T)

print("\nDetailed Results:")
print(df)


/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed

Running backtest on DMART.NS



/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed

Running backtest on TCS.NS
Running backtest on BEL.NS



/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed

...

Running backtest on HDFCLIFE.NS
Running backtest on MAXHEALTH
No data for MAXHEALTH, skipping.
Running backtest on SBIL


/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SBIL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2022-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1641013200, endDate = 1672462800")')
/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed


No data for SBIL, skipping.
Running backtest on TRENT.NS
Running backtest on VBL.NS


/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KPIL']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GMRTHU']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-3377890638.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfina

Running backtest on KPIL
No data for KPIL, skipping.
Running backtest on GMRTHU
No data for GMRTHU, skipping.
Running backtest on SWIGGY
No data for SWIGGY, skipping.
Running backtest on ETERNAL
No data for ETERNAL, skipping.
Running backtest on PAYTM
No data for PAYTM, skipping.
Running backtest on ADANPOWER
No data for ADANPOWER, skipping.
Running backtest on HEG
No data for HEG, skipping.
Aggregate Summary:
      average_loss  average_trade_pnl  average_win  final_value  gross_loss  \
Mean    -19.797729          -0.806412    14.127954  9996.016447  -77.263432   
Sum            NaN                NaN          NaN          NaN         NaN   
Max            NaN                NaN          NaN          NaN         NaN   

      gross_profit  losing_trades  max_drawdown  max_loss  max_win  \
Mean     73.279878            NaN      0.735882       NaN      NaN   
Sum            NaN           31.0           NaN       NaN      NaN   
Max            NaN            NaN           NaN       0.0  

In [11]:
import backtrader as bt
import yfinance as yf
import pandas as pd


class CustomStrategy(bt.Strategy):
    params = dict(rsi_period=14, ma_periods=(10, 20, 50), vol_period=20)

    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)
        self.ma10 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[0])
        self.ma20 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[1])
        self.ma50 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[2])
        self.volma = bt.indicators.SMA(self.data.volume, period=self.p.vol_period)

    def next(self):
        score = 0
        if self.rsi[0] < 30:
            score += 2
        elif self.rsi[0] > 70:
            score -= 2
        elif 45 <= self.rsi[0] <= 55:
            score += 1

        if self.ma10[0] > self.ma20[0] > self.ma50[0]:
            score += 2
        elif self.ma10[0] < self.ma20[0] < self.ma50[0]:
            score -= 2

        if self.data.volume[0] > 1.5 * self.volma[0]:
            score += 1

        if score >= 3:
            signal = "BUY"
        elif score <= -2:
            signal = "SELL"
        else:
            signal = "HOLD"

        if not self.position:
            if signal == "BUY":
                self.buy()
        else:
            if signal in ("SELL", "HOLD"):
                self.sell()


tickers = [
    "DMART.NS", "TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS",
    "HDFCLIFE.NS", "MAXHEALTH", "SBILIFE", "TRENT.NS", "VBL.NS",
    "KPIL", "GMRTHU", "SWIGGY", "ETERNAL", "PAYTM", "ADANPOWER", "HEG"
]

results = []

for ticker in tickers:
    print(f"Running backtest on {ticker}")
    data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
    if data.empty:
        print(f"No data for {ticker}, skipping.")
        continue

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(1)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(CustomStrategy)
    data_feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_feed)
    cerebro.broker.setcash(10000)

    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade")

    strat = cerebro.run()[0]

    trade_analysis = strat.analyzers.trade.get_analysis()
    total_trades = trade_analysis.get("total", {}).get("closed", 0)

    perf = dict(
        ticker=ticker,
        final_value=cerebro.broker.getvalue(),
        sharpe=strat.analyzers.sharpe.get_analysis().get("sharperatio"),
        sqn=strat.analyzers.sqn.get_analysis().get("sqn"),
        max_drawdown=strat.analyzers.drawdown.get_analysis().max.drawdown,
        total_trades=total_trades,
    )

    results.append(perf)

df = pd.DataFrame(results)
print(df)



/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed

Running backtest on DMART.NS
Running backtest on TCS.NS
Running backtest on BEL.NS



/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed

Running backtest on DLF.NS
Running backtest on GODREJCP.NS



/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")


Running backtest on HDFCLIFE.NS


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MAXHEALTH']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SBILIFE']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[****

Running backtest on MAXHEALTH
No data for MAXHEALTH, skipping.
Running backtest on SBILIFE
No data for SBILIFE, skipping.
Running backtest on TRENT.NS
Running backtest on VBL.NS



/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KPIL']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GMRTHU']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-3946357873.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfin

Running backtest on KPIL
No data for KPIL, skipping.
Running backtest on GMRTHU
No data for GMRTHU, skipping.
Running backtest on SWIGGY
No data for SWIGGY, skipping.
Running backtest on ETERNAL
No data for ETERNAL, skipping.
Running backtest on PAYTM
No data for PAYTM, skipping.
Running backtest on ADANPOWER
No data for ADANPOWER, skipping.
Running backtest on HEG
No data for HEG, skipping.
        ticker   final_value sharpe       sqn  max_drawdown  total_trades
0     DMART.NS  10071.500000   None  3.917196      0.561290             3
1       TCS.NS   9797.967193   None -1.405450      2.175242             6
2       BEL.NS   9993.253945   None -0.956622      0.086605             9
3       DLF.NS  10034.308535   None  0.812123      0.276549             9
4  GODREJCP.NS   9939.432545   None -0.908731      1.192048            16
5  HDFCLIFE.NS   9999.085397   None -0.018173      0.562019             8
6     TRENT.NS   9984.506034   None -0.186632      0.927600             7
7       VBL.N

In [3]:
pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.8 MB/s eta 0:00:00


**Overall PnL of Backtester 3**

In [19]:
import backtrader as bt
import yfinance as yf
import pandas as pd


class CustomStrategy(bt.Strategy):
    params = dict(rsi_period=14, ma_periods=(10, 20, 50), vol_period=20)

    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)
        self.ma10 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[0])
        self.ma20 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[1])
        self.ma50 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[2])
        self.volma = bt.indicators.SMA(self.data.volume, period=self.p.vol_period)

    def next(self):
        score = 0
        if self.rsi[0] < 30:
            score += 2
        elif self.rsi[0] > 70:
            score -= 2
        elif 45 <= self.rsi[0] <= 55:
            score += 1

        if self.ma10[0] > self.ma20[0] > self.ma50[0]:
            score += 2
        elif self.ma10[0] < self.ma20[0] < self.ma50[0]:
            score -= 2

        if self.data.volume[0] > 1.5 * self.volma[0]:
            score += 1

        if score >= 3:
            signal = "BUY"
        elif score <=  -2:
            signal = "SELL"
        else:
            signal = "HOLD"

        if not self.position:
            if signal == "BUY":
                self.buy()
        else:
            if signal in ("SELL", "HOLD"):
                self.sell()


tickers = [
    "DMART.NS", "TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS",
    "HDFCLIFE.NS", "MAXHEALTH", "SBIL", "TRENT.NS", "VBL.NS",
    "KPIL", "GMRTHU", "SWIGGY", "ETERNAL", "PAYTM", "ADANPOWER", "HEG"
]

results = []

for ticker in tickers:
    print(f"Running backtest on {ticker}")
    data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
    if data.empty:
        print(f"No data for {ticker}, skipping.")
        continue

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(1)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(CustomStrategy)
    data_feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_feed)
    cerebro.broker.setcash(10000)

    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade")

    strat = cerebro.run()[0]

    trade_analysis = strat.analyzers.trade.get_analysis()

    total_trades = trade_analysis.get("total", {}).get("closed", 0)
    won_trades = trade_analysis.get("won", {}).get("total", 0)
    lost_trades = trade_analysis.get("lost", {}).get("total", 0)
    win_rate = won_trades / total_trades if total_trades != 0 else 0
    gross_profit = trade_analysis.get("won", {}).get("pnl", {}).get("total", 0)
    gross_loss = trade_analysis.get("lost", {}).get("pnl", {}).get("total", 0)
    net_profit = trade_analysis.get("pnl", {}).get("net", {}).get("total", 0)
    avg_win = trade_analysis.get("won", {}).get("pnl", {}).get("average", 0)
    avg_loss = trade_analysis.get("lost", {}).get("pnl", {}).get("average", 0)
    avg_trade_pnl = net_profit / total_trades if total_trades != 0 else 0
    max_win = trade_analysis.get("won", {}).get("pnl", {}).get("max", 0)
    max_loss = trade_analysis.get("lost", {}).get("pnl", {}).get("min", 0)
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float("inf")

    max_drawdown = strat.analyzers.drawdown.get_analysis().max.drawdown
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get("sharperatio", None)
    sqn = strat.analyzers.sqn.get_analysis().get("sqn", None)

    results.append(
        {
            "ticker": ticker,
            "final_value": cerebro.broker.getvalue(),
            "sharpe": sharpe_ratio,
            "sqn": sqn,
            "max_drawdown": max_drawdown,
            "total_trades": total_trades,
            "winning_trades": won_trades,
            "losing_trades": lost_trades,
            "win_rate": win_rate,
            "gross_profit": gross_profit,
            "gross_loss": gross_loss,
            "net_profit": net_profit,
            "average_win": avg_win,
            "average_loss": avg_loss,
            "average_trade_pnl": avg_trade_pnl,
            "max_win": max_win,
            "max_loss": max_loss,
            "profit_factor": profit_factor,
        }
    )

df = pd.DataFrame(results)

# Calculate aggregates
mean_cols = [
    "final_value",
    "sqn",
    "max_drawdown",
    "win_rate",
    "gross_profit",
    "gross_loss",
    "average_win",
    "average_loss",
    "average_trade_pnl",
]
sum_cols = ["total_trades", "winning_trades", "losing_trades", "net_profit"]
max_cols = ["max_win", "max_loss"]

aggregate_summary = pd.DataFrame(
    {
        "Mean": df[mean_cols].mean(),
        "Sum": df[sum_cols].sum(),
        "Max": df[max_cols].max(),
    }
)

print("Aggregate Summary:")
print(aggregate_summary.T)

print("\nDetailed Results:")
print(df)


/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed

Running backtest on DMART.NS
Running backtest on TCS.NS



/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed


Running backtest on BEL.NS
Running backtest on DLF.NS


/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed

Running backtest on GODREJCP.NS
Running backtest on HDFCLIFE.NS



/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MAXHEALTH']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SBIL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-01-01 -> 2024-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1704085200, endDate = 1735621200")')
/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.downloa

Running backtest on MAXHEALTH
No data for MAXHEALTH, skipping.
Running backtest on SBIL
No data for SBIL, skipping.
Running backtest on TRENT.NS
Running backtest on VBL.NS


/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KPIL']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-1757942829.py:58: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GMRTHU']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipytho

Running backtest on KPIL
No data for KPIL, skipping.
Running backtest on GMRTHU
No data for GMRTHU, skipping.
Running backtest on SWIGGY
No data for SWIGGY, skipping.
Running backtest on ETERNAL
No data for ETERNAL, skipping.
Running backtest on PAYTM
No data for PAYTM, skipping.
Running backtest on ADANPOWER
No data for ADANPOWER, skipping.
Running backtest on HEG
No data for HEG, skipping.
Aggregate Summary:
      average_loss  average_trade_pnl  average_win   final_value  gross_loss  \
Mean    -59.646975          12.817722    66.107384  10145.745026 -263.023786   
Sum            NaN                NaN          NaN           NaN         NaN   
Max            NaN                NaN          NaN           NaN         NaN   

      gross_profit  losing_trades  max_drawdown  max_loss     max_win  \
Mean    410.890793            NaN      1.993567       NaN         NaN   
Sum            NaN           38.0           NaN       NaN         NaN   
Max            NaN            NaN           Na

In [8]:
import backtrader as bt
import yfinance as yf
import pandas as pd

class CustomStrategy(bt.Strategy):
    params = dict(rsi_period=14, ma_periods=(10,20,50), vol_period=20)

    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)
        self.ma10 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[0])
        self.ma20 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[1])
        self.ma50 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[2])
        self.volma = bt.indicators.SMA(self.data.volume, period=self.p.vol_period)
        self.trades = []  # list to collect trades info

    def log(self, txt):
        dt = self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}, Size: {order.executed.size}')
                self.trade_entry_date = self.datas[0].datetime.date(0)
                self.trade_entry_price = order.executed.price
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}, Size: {order.executed.size}')
                trade_exit_date = self.datas[0].datetime.date(0)
                trade_exit_price = order.executed.price
                profit = (trade_exit_price - self.trade_entry_price) * order.executed.size
                self.trades.append({
                    'entry_date': self.trade_entry_date,
                    'entry_price': self.trade_entry_price,
                    'exit_date': trade_exit_date,
                    'exit_price': trade_exit_price,
                    'profit': profit
                })

    def next(self):
        score = 0
        if self.rsi[0] < 30:
            score += 2
        elif self.rsi[0] > 70:
            score -= 2
        elif 45 <= self.rsi[0] <= 55:
            score += 1

        if self.ma10[0] > self.ma20[0] > self.ma50[0]:
            score += 2
        elif self.ma10[0] < self.ma20[0] < self.ma50[0]:
            score -= 2

        if self.data.volume[0] > 1.5 * self.volma[0]:
            score += 1

        if not self.position:
            if score >=3:
                self.buy()
        else:
            if score <= -2:
                self.sell()

# Run backtests for multiple tickers and collect trade details

tickers = ["DMART.NS","TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS", "HDFCLIFE.NS",
    "MAXHEALTH.NS", "SBILIFE.NS", "TRENT.NS", "VBL.NS", "KPIL.NS", "GMRPUI.NS","SWIGGY.NS","ETERNAL.NS","PAYTM.NS","ADANIPOWER.NS","HEG.NS","PAYTM.NS"]
trade_details = []

for ticker in tickers:
    print(f'Running backtest for {ticker}...')
    data = yf.download(ticker, start='2022-01-01', end='2023-01-01')
    if data.empty:
        print(f'No data for {ticker}, skipping.')
        continue

    if isinstance(data.columns, pd.MultiIndex):
        # flatten columns
        data.columns = data.columns.get_level_values(0)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(CustomStrategy)
    data_feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_feed)
    cerebro.broker.setcash(10000)

    results = cerebro.run()
    strat = results[0]

    print(f'Final Portfolio Value for {ticker}: {cerebro.broker.getvalue():.2f}')
    trade_details.extend([{'Ticker': ticker, **t} for t in strat.trades])

# Convert trade details to DataFrame
trade_df = pd.DataFrame(trade_details)
print(trade_df)

/tmp/ipython-input-1087200136.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2022-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed

Running backtest for DMART.NS...
2022-09-12, BUY EXECUTED, Price: 4440.00, Size: 1
2022-10-19, SELL EXECUTED, Price: 4168.00, Size: -1
Final Portfolio Value for DMART.NS: 9728.00
Running backtest for TCS.NS...



/tmp/ipython-input-1087200136.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2022-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1087200136.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2022-01-01', end='2023-01-01')


2022-04-08, BUY EXECUTED, Price: 3489.66, Size: 1
2022-05-02, SELL EXECUTED, Price: 3328.79, Size: -1
2022-08-24, BUY EXECUTED, Price: 3142.51, Size: 1
2022-09-16, SELL EXECUTED, Price: 2936.32, Size: -1
2022-11-14, BUY EXECUTED, Price: 3181.27, Size: 1
Final Portfolio Value for TCS.NS: 9568.53
Running backtest for BEL.NS...


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1087200136.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2022-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed

...

2022-08-30, BUY EXECUTED, Price: 60.20, Size: 1
2022-12-19, SELL EXECUTED, Price: 62.40, Size: -1
Final Portfolio Value for ETERNAL.NS: 10002.20
Running backtest for PAYTM.NS...
2022-06-09, BUY EXECUTED, Price: 611.80, Size: 1
2022-09-20, SELL EXECUTED, Price: 710.10, Size: -1
Final Portfolio Value for PAYTM.NS: 10098.30
Running backtest for ADANIPOWER.NS...


/tmp/ipython-input-1087200136.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2022-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1087200136.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2022-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed

2022-03-24, BUY EXECUTED, Price: 26.14, Size: 1
2022-05-24, SELL EXECUTED, Price: 68.19, Size: -1
2022-05-25, BUY EXECUTED, Price: 61.82, Size: 1
2022-10-13, SELL EXECUTED, Price: 71.39, Size: -1
Final Portfolio Value for ADANIPOWER.NS: 10051.62
Running backtest for HEG.NS...



/tmp/ipython-input-1087200136.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2022-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed

2022-04-18, BUY EXECUTED, Price: 248.21, Size: 1
2022-05-18, SELL EXECUTED, Price: 206.24, Size: -1
2022-07-28, BUY EXECUTED, Price: 204.00, Size: 1
2022-10-03, SELL EXECUTED, Price: 203.14, Size: -1
Final Portfolio Value for HEG.NS: 9957.17
Running backtest for PAYTM.NS...
2022-06-09, BUY EXECUTED, Price: 611.80, Size: 1
2022-09-20, SELL EXECUTED, Price: 710.10, Size: -1
Final Portfolio Value for PAYTM.NS: 10098.30
           Ticker  entry_date  entry_price   exit_date   exit_price  \
0        DMART.NS  2022-09-12  4440.000000  2022-10-19  4168.000000   
1          TCS.NS  2022-04-08  3489.656884  2022-05-02  3328.791918   
2          TCS.NS  2022-08-24  3142.509795  2022-09-16  2936.318407   
3          BEL.NS  2022-03-16    66.369414  2022-12-16    98.703363   
4          DLF.NS  2022-04-20   368.282422  2022-05-16   311.922551   
5          DLF.NS  2022-08-23   353.649664  2022-10-12   352.964103   
6     GODREJCP.NS  2022-04-26   741.723226  2022-06-13   715.578950   
7     GODREJ

In [29]:
import backtrader as bt
import yfinance as yf
import pandas as pd


class CustomStrategy(bt.Strategy):
    params = dict(rsi_period=14, ma_periods=(10, 20, 50), vol_period=20)

    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)
        self.ma10 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[0])
        self.ma20 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[1])
        self.ma50 = bt.indicators.SMA(self.data.close, period=self.p.ma_periods[2])
        self.volma = bt.indicators.SMA(self.data.volume, period=self.p.vol_period)
        self.trades = []

    def log(self, txt):
        dt = self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}, Size: {order.executed.size}')
                self.trade_entry_date = self.datas[0].datetime.date(0)
                self.trade_entry_price = order.executed.price
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}, Size: {order.executed.size}')
                trade_exit_date = self.datas[0].datetime.date(0)
                trade_exit_price = order.executed.price
                profit = (trade_exit_price - self.trade_entry_price) * order.executed.size
                self.trades.append({
                    'entry_date': self.trade_entry_date,
                    'entry_price': self.trade_entry_price,
                    'exit_date': trade_exit_date,
                    'exit_price': trade_exit_price,
                    'profit': profit
                })

    def next(self):
        if len(self) < max(self.p.ma_periods):  # Wait for indicators to initialize
            return

        score = 0
        if self.rsi[0] < 30:
            score += 2
        elif self.rsi[0] > 70:
            score -= 2
        elif 45 <= self.rsi[0] <= 55:
            score += 1

        if self.ma10[0] > self.ma20[0] > self.ma50[0]:
            score += 2
        elif self.ma10[0] < self.ma20[0] < self.ma50[0]:
            score -= 2

        if self.data.volume[0] > 1.5 * self.volma[0]:
            score += 1

        if score >= 3:
            signal = "BUY"
        elif score <= -2:
            signal = "SELL"
        else:
            signal = "HOLD"

        if not self.position:
            if signal == "BUY":
                self.buy()
        else:
            if signal in ("SELL", "HOLD"):
                self.sell()


# Backtest multiple tickers
tickers = [
    "DMART.NS", "TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS",
    "HDFCLIFE.NS", "MAXHEALTH", "SBIL", "TRENT.NS", "VBL.NS",
    "KPIL", "GMRTHU", "SWIGGY", "ETERNAL", "PAYTM", "ADANPOWER", "HEG"
]

all_trade_details = []

for ticker in tickers:
    print(f'Running backtest for {ticker}...')
    data = yf.download(ticker, start='2024-01-01', end='2024-12-31')

    if data.empty or len(data) < max(50, 20):  # Check sufficient data size for indicators
        print(f'Insufficient data for {ticker}, skipping.')
        continue

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(CustomStrategy)
    data_feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_feed)
    cerebro.broker.setcash(10000)

    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')

    strat = cerebro.run()[0]

    print(f'Final Portfolio Value for {ticker}: {cerebro.broker.getvalue()}')

    all_trade_details.extend([{'Ticker': ticker, **t} for t in strat.trades])

# Convert trade details to DataFrame
trade_df = pd.DataFrame(all_trade_details)
print(trade_df)


/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed

Running backtest for DMART.NS...
2024-03-18, BUY EXECUTED, Price: 3932.10, Size: 1
2024-03-19, SELL EXECUTED, Price: 4008.00, Size: -1
2024-03-22, BUY EXECUTED, Price: 4159.80, Size: 1
2024-03-26, SELL EXECUTED, Price: 4315.75, Size: -1
2024-04-29, BUY EXECUTED, Price: 4560.50, Size: 1
2024-05-02, SELL EXECUTED, Price: 4609.95, Size: -1
2024-05-15, BUY EXECUTED, Price: 4614.95, Size: 1
2024-05-22, SELL EXECUTED, Price: 4763.15, Size: -1
2024-05-23, BUY EXECUTED, Price: 4794.50, Size: 1
2024-05-24, SELL EXECUTED, Price: 4797.90, Size: -1
2024-05-27, BUY EXECUTED, Price: 4651.55, Size: 1
2024-05-28, SELL EXECUTED, Price: 4551.90, Size: -1
2024-06-26, BUY EXECUTED, Price: 4775.00, Size: 1
2024-06-28, SELL EXECUTED, Price: 4900.00, Size: -1
2024-07-01, BUY EXECUTED, Price: 4725.10, Size: 1
2024-07-08, SELL EXECUTED, Price: 4860.00, Size: -1
2024-07-16, BUY EXECUTED, Price: 5000.00, Size: 1
2024-07-19, SELL EXECUTED, Price: 5062.10, Size: -1
2024-08-01, BUY EXECUTED, Price: 4949.00, Size: 1


/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed


2024-03-15, BUY EXECUTED, Price: 193.99, Size: 1
2024-03-19, SELL EXECUTED, Price: 187.34, Size: -1
2024-05-08, BUY EXECUTED, Price: 224.82, Size: 1
2024-05-09, SELL EXECUTED, Price: 232.04, Size: -1
2024-05-10, BUY EXECUTED, Price: 225.51, Size: 1
2024-05-13, SELL EXECUTED, Price: 226.00, Size: -1
2024-06-05, BUY EXECUTED, Price: 261.73, Size: 1
2024-06-10, SELL EXECUTED, Price: 285.87, Size: -1
2024-07-19, BUY EXECUTED, Price: 306.65, Size: 1
2024-07-25, SELL EXECUTED, Price: 294.82, Size: -1
2024-11-25, BUY EXECUTED, Price: 287.74, Size: 1
2024-11-26, SELL EXECUTED, Price: 291.70, Size: -1
2024-12-05, BUY EXECUTED, Price: 311.55, Size: 1
2024-12-06, SELL EXECUTED, Price: 312.04, Size: -1
2024-12-19, BUY EXECUTED, Price: 292.70, Size: 1
2024-12-20, SELL EXECUTED, Price: 295.67, Size: -1
Final Portfolio Value for BEL.NS: 10020.823564704287
Running backtest for DLF.NS...
2024-03-14, BUY EXECUTED, Price: 815.34, Size: 1
2024-03-15, SELL EXECUTED, Price: 825.81, Size: -1
2024-04-05, BUY 

/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MAXHEALTH']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SBIL']: YFPricesMissingError('possibly delisted; no price data found  (1d

2024-07-31, BUY EXECUTED, Price: 1419.65, Size: 1
2024-08-02, SELL EXECUTED, Price: 1438.77, Size: -1
2024-08-05, BUY EXECUTED, Price: 1398.58, Size: 1
2024-08-07, SELL EXECUTED, Price: 1465.19, Size: -1
2024-08-09, BUY EXECUTED, Price: 1447.01, Size: 1
2024-08-12, SELL EXECUTED, Price: 1416.52, Size: -1
2024-09-04, BUY EXECUTED, Price: 1431.74, Size: 1
2024-09-05, SELL EXECUTED, Price: 1456.34, Size: -1
2024-09-06, BUY EXECUTED, Price: 1430.26, Size: 1
2024-09-12, SELL EXECUTED, Price: 1484.58, Size: -1
2024-09-16, BUY EXECUTED, Price: 1452.40, Size: 1
2024-09-19, SELL EXECUTED, Price: 1416.29, Size: -1
2024-09-20, BUY EXECUTED, Price: 1430.75, Size: 1
2024-09-25, SELL EXECUTED, Price: 1422.88, Size: -1
2024-12-10, BUY EXECUTED, Price: 1116.23, Size: 1
2024-12-11, SELL EXECUTED, Price: 1123.04, Size: -1
Final Portfolio Value for GODREJCP.NS: 10190.282725293036
Running backtest for HDFCLIFE.NS...
2024-03-22, BUY EXECUTED, Price: 620.07, Size: 1
2024-03-26, SELL EXECUTED, Price: 634.93,


/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KPIL']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed

2024-03-27, BUY EXECUTED, Price: 3876.48, Size: 1
2024-04-03, SELL EXECUTED, Price: 3878.98, Size: -1
2024-04-18, BUY EXECUTED, Price: 3981.82, Size: 1
2024-04-24, SELL EXECUTED, Price: 4151.85, Size: -1
2024-04-30, BUY EXECUTED, Price: 4443.10, Size: 1
2024-05-02, SELL EXECUTED, Price: 4403.06, Size: -1
2024-06-03, BUY EXECUTED, Price: 4741.69, Size: 1
2024-06-04, SELL EXECUTED, Price: 4680.94, Size: -1
2024-06-05, BUY EXECUTED, Price: 4679.95, Size: 1
2024-06-07, SELL EXECUTED, Price: 4874.23, Size: -1
2024-07-19, BUY EXECUTED, Price: 5303.91, Size: 1
2024-07-22, SELL EXECUTED, Price: 5105.58, Size: -1
2024-08-08, BUY EXECUTED, Price: 5510.23, Size: 1
2024-08-13, SELL EXECUTED, Price: 6369.48, Size: -1
2024-10-07, BUY EXECUTED, Price: 7300.88, Size: 1
2024-10-08, SELL EXECUTED, Price: 7513.49, Size: -1
2024-10-18, BUY EXECUTED, Price: 7713.32, Size: 1
2024-10-28, SELL EXECUTED, Price: 7355.88, Size: -1
2024-11-08, BUY EXECUTED, Price: 6544.33, Size: 1
2024-11-11, SELL EXECUTED, Price


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GMRTHU']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SWIGGY']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ETERNAL']: YFTzMissingError('possibly delisted; no timezone found')
/tmp/ipython-input-1328127973.py:86: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tic

Insufficient data for GMRTHU, skipping.
Running backtest for SWIGGY...
Insufficient data for SWIGGY, skipping.
Running backtest for ETERNAL...
Insufficient data for ETERNAL, skipping.
Running backtest for PAYTM...
Insufficient data for PAYTM, skipping.
Running backtest for ADANPOWER...
Insufficient data for ADANPOWER, skipping.
Running backtest for HEG...
Insufficient data for HEG, skipping.
      Ticker  entry_date  entry_price   exit_date   exit_price      profit
0   DMART.NS  2024-03-18  3932.100098  2024-03-19  4008.000000  -75.899902
1   DMART.NS  2024-03-22  4159.799805  2024-03-26  4315.750000 -155.950195
2   DMART.NS  2024-04-29  4560.500000  2024-05-02  4609.950195  -49.450195
3   DMART.NS  2024-05-15  4614.950195  2024-05-22  4763.149902 -148.199707
4   DMART.NS  2024-05-23  4794.500000  2024-05-24  4797.899902   -3.399902
..       ...         ...          ...         ...          ...         ...
87    VBL.NS  2024-09-26   631.498498  2024-09-30   622.224980    9.273518
88   

In [31]:
trade_df.profit.sum()

np.float64(-1182.9360562177526)